In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import csv
import re
import os
from urllib.error import HTTPError
from astropy.time import Time
from astropy.coordinates import SkyCoord
import astropy.units as u


In [13]:
# Move up one directory from ia_supernovae_stuff to Query_VLASS_2
%cd ..

# Now you should be in Query_VLASS_2, you can import the module
import Query_VLASS_2.vlass_search_editor as vs

# After importing, if you want to go back to your working folder:
%cd ia_supernovae_stuff

d:\summer_research_code\Query_VLASS_2
File downloaded to: VLASS_dyn_summary.php
d:\summer_research_code\Query_VLASS_2\ia_supernovae_stuff


### Function to Save NumPy Arrays from Cutouts

NameError: name 'tilename' is not defined

In [1]:
coord_str = "13:14:25.45 +50:58:39.7"
c = SkyCoord(coord_str, unit=(u.hourangle, u.deg))

# Example transient name and epoch
name = "ZTF18aagrtxs"
epoch = "VLASS2.1"


NameError: name 'SkyCoord' is not defined

### Function to Get Unique Part of Fits File

In [8]:
import os
from urllib.request import urlopen
from contextlib import closing
import numpy as np
from astropy.coordinates import SkyCoord

def get_unique_fits_part(name, epoch, c):
    """
    Retrieves the unique part of the FITS file download link for a given source name, epoch, and coordinates.

    Parameters
    ----------
    name : str
        The name of the source.
    epoch : str
        The epoch of the observation (e.g., 'VLASS3.2').
    coordinates : SkyCoord
        The coordinates as a SkyCoord object.

    Returns
    -------
    str
        The unique part of the FITS download link or a message indicating failure.
    """
    tiles = vs.get_tiles()
    tilenames, epochs, obsdates = vs.search_tiles(tiles, c)
    
    # Assume that it is the same tile name regardless of epoch 
    tile = tilenames[0]

    subtiles, c_tiles = vs.get_subtiles(tile, epoch)
    print("Tile Found:")
    print(tile, epoch)
    
    # Find angular separation from the tiles to the location
    dist = c.separation(c_tiles)
    # Find tile with the smallest separation 
    subtile = subtiles[np.argmin(dist)]
    
    # Unique part of the link (e.g., tile and file name)
    unique_part = "%s/%s.I.iter1.image.pbcor.tt0.subim.fits" % (tile, subtile[:-1])
    
    return unique_part  # Return the unique part of the link



In [24]:
from astropy.coordinates import SkyCoord
from astropy import units as u

# Example coordinates
coord_str = "13:14:25.45 +50:58:39.7"
c = SkyCoord(coord_str, unit=(u.hourangle, u.deg))

# Example transient name and epoch
name = "ZTF18aagrtxs"
epoch = "VLASS2.1"

get_unique_fits_part(name, epoch, c)

https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T23t14/
Tile Found:
T23t14 VLASS2.1


'T23t14/VLASS2.1.ql.T23t14.J131533+503000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits'

In [9]:
df = pd.read_csv("final_ia_transients.csv")

In [37]:
import pandas as pd
from astropy.coordinates import SkyCoord

def fits_in_csv(csv_name, start_row=0, chunk_size=100):
    # Step 1: Read the CSV
    df = pd.read_csv(csv_name)

    # Step 2: Check if the "Unique FITS Part" column already exists
    if "Unique FITS Part" not in df.columns:
        df["Unique FITS Part"] = None  # Initialize with None if not present

    # Step 3: Initialize an empty list to store the unique FITS parts
    unique_fits_parts = []

    # Step 4: Define the end row for processing
    end_row = min(start_row + chunk_size, len(df))

    # Step 5: Loop through the specified rows in the DataFrame
    for index in range(start_row, end_row):
        row = df.iloc[index]
        name = row["Transient Name"]
        epoch = row["Epoch"]
        ra = row["RA"]  # Replace with the actual column name for RA
        dec = row["Dec"]  # Replace with the actual column name for Dec
        
        # Print the transient name being processed
        print(f"Processing Transient: {name}")

        # Step 6: Check if the FITS part is already computed
        if pd.isna(row["Unique FITS Part"]):
            try:
                # Print RA and Dec values for debugging
                print(f"RA: {ra}, Dec: {dec}")

                # Convert RA and Dec into a SkyCoord object
                c = SkyCoord(ra=ra, dec=dec, unit=(u.hourangle, u.deg))  # Make sure RA/Dec are in degrees

                # Get the unique part of the FITS link
                print(f"Name: {name}")
                print(f"Epoch: {epoch}")
                print(f"Skycoord:{c}")
                fits_part = get_unique_fits_part(name, epoch, c)
                
                # Print the constructed FITS URL (for debugging)
                print(f"Constructed FITS URL: {fits_part}")

                # Append the FITS part to the list
                unique_fits_parts.append(fits_part)
            except Exception as e:
                print(f"Error processing {name} (RA: {ra}, Dec: {dec}): {e}")
                unique_fits_parts.append(None)  # Append None if there's an error
        else:
            unique_fits_parts.append(row["Unique FITS Part"])  # Keep existing FITS part

    # Step 7: Update the DataFrame with the new unique FITS parts
    df.loc[start_row:end_row-1, "Unique FITS Part"] = unique_fits_parts

    # Step 8: Save the updated DataFrame to the original CSV file
    df.to_csv(csv_name, index=False)

    print(f"Processed rows {start_row} to {end_row} and updated {csv_name}")

# Example usage:
fits_in_csv("final_ia_transients.csv", start_row=4800, chunk_size=200)

fits_in_csv("final_ia_transients.csv", start_row=5000, chunk_size=200)

fits_in_csv("final_ia_transients.csv", start_row=5200, chunk_size=200)


Processing Transient: ZTF20aabcouv
RA: 03:45:22.60, Dec: +04:16:13.1
Name: ZTF20aabcouv
Epoch: VLASS1.1v2
Skycoord:<SkyCoord (ICRS): (ra, dec) in deg
    (56.34416667, 4.27030556)>
https://archive-new.nrao.edu/vlass/quicklook/VLASS1.1v2/T12t06/
Tile Found:
T12t06 VLASS1.1v2
Constructed FITS URL: T12t06/VLASS1.1.ql.T12t06.J034603+043000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
Processing Transient: ZTF20aabcouv
RA: 03:45:22.60, Dec: +04:16:13.1
Name: ZTF20aabcouv
Epoch: VLASS2.1
Skycoord:<SkyCoord (ICRS): (ra, dec) in deg
    (56.34416667, 4.27030556)>
https://archive-new.nrao.edu/vlass/quicklook/VLASS2.1/T12t06/
Tile Found:
T12t06 VLASS2.1
Constructed FITS URL: T12t06/VLASS2.1.ql.T12t06.J034603+043000.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits
Processing Transient: ZTF20aabcouv
RA: 03:45:22.60, Dec: +04:16:13.1
Name: ZTF20aabcouv
Epoch: VLASS3.1
Skycoord:<SkyCoord (ICRS): (ra, dec) in deg
    (56.34416667, 4.27030556)>
https://archive-new.nrao.edu/vlass/quicklook/VLASS3.1/T12t0